In [1]:
using Pkg
using DotEnv
include("src/test_name.jl")
using .Test_name

┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.to_rcm_time :: Tuple{String}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.get_bbox :: Tuple{Any}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.bounding_box_to_geojson :: Tuple{}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.coordinate_to_geojson :: Union{Tuple{T}, Tuple{T, T}} where T<:Real` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.wkt_to_tuples :: Tuple{Any}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243


In [2]:
DotEnv.config();

# metadata

In [3]:
access_token = Test_name.Download.get_access_token_copernicus(username = get(ENV, "DATASPACE_USERNAME", ""), password = get(ENV, "DATASPACE_PASSWORD", ""));

In [4]:
access_token

"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3MDM3NzI2MjYsImlhdCI6MTcwMzc3MjAyNiwianRpIjoiMjQyOTA2YTEtNzFiOS00ZTRlLThkNWMtMmY3NjUwZWM3YjNhIiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZX" ⋯ 1869 bytes ⋯ "KbWHxUsoi5uBXzj04Y82qmpfhybLg_h-o9NDSg2OpslarxnEcobn6mDg87kYKIsKkGwoVd43WyL9nHLhBH9uZlZg_LNlvDlYY3ZB4j10Rl0qqxpAcobmi3TSfiPAJISIvo3HtZ5cT7jv8S6fasVpwyx1p694OOlbcVve-3sy8rXLwoRZXp8XOHrc6XJrJC3XDPAM1lc7bw4D9r4_oDsm6aaIp5gm2mpgx6j9xDnN0HEHfIeEsPBrASdb8wil9vys468p-O2seKYoQQ"

In [ ]:
lat = 64.1743
lon = 51.73

In [ ]:
bbox = Test_name.Download.Metadata.get_bbox([lat, lon])
json = Test_name.Download.Metadata.coordinate_to_geojson(lat, lon)

In [ ]:
using Test

# Test for get_bbox function

# Test with valid inputs
@test get_bbox([10, 20]) == "10.0,20.0,10.0,20.0"
@test get_bbox([10, 20, 30, 40]) == "10.0,20.0,30.0,40.0"
@test get_bbox("10,20") == "10.0,20.0,10.0,20.0"
@test get_bbox("10,20,30,40") == "10.0,20.0,30.0,40.0"

# Additional tests as previously provided

println("All tests passed!")


In [ ]:
area = get_bbox([lat,lon])

In [ ]:
data = Test_name.Download.Metadata.download_metadata_copernicus(bbox=area,
start_datetime="2023-11-01T00:00:00Z", 
end_datetime="2023-11-26T23:59:59Z", 
max_records="999",
collection="Sentinel1",
product_type= "GRD",
verbose=1);

df = Test_name.Download.Metadata.vector_of_dicts_to_dataframe(data["features"]);


In [ ]:
size(df)

In [ ]:
Test_name.Download.Metadata.plot_polygons(df, "geometry_coordinates_1", "properties_updated")


In [ ]:
df

 Download

In [ ]:
#Test_name.Download.Products.download_product_copernicus(access_token,df.id[2],"/Users/kaaso/Documents/coding/JuliaEO2024/data/raw/$(df.id[2]).zip")

# Load Image

In [ ]:
safe_folder = "/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE"

In [ ]:
test_img = Test_name.Sensors.Sentinel1.Sentinel1GRD(safe_folder)

In [ ]:
Test_name.Visualize.sar2gray(test_img.data[1],p_quantile=0.99)

In [ ]:
Test_name.Visualize.sar2gray(test_img.data[1][13000:13500,20000:20350],p_quantile=0.99)

In [ ]:
Test_name.Visualize.sar2gray(test_img.data[1][13200:13300,20060:20150],p_quantile=0.99)

## Stats

In [ ]:
Test_name.Analysis.image_statistics(test_img.data[1])

## Coordinate

let say we found ships in these coordinates:

In [ ]:
rows = [13250, 20080, 13120, 10,30,1000,10023,5212,1233,2222];
columns = [20080, 13250, 5510,10,30,1000,10023,5212,1233,2222];

In [ ]:
struct CubicSpline
    x::Vector{Float64}
    y::Vector{Float64}
    a::Vector{Float64}
    b::Vector{Float64}
    c::Vector{Float64}
    d::Vector{Float64}
end

function calculateCubicSplineCoefficients(x, y)
    n = length(x)
    a = copy(y)

    # Calculate the coefficients b, c, d
    b = zeros(n)
    d = zeros(n)
    h = diff(x)
    alpha = [0; 3 ./ h[2:end] .* (a[3:end] - a[2:end-1]) - 3 ./ h[1:end-1] .* (a[2:end-1] - a[1:end-2])]

    c = zeros(n)
    l = zeros(n)
    mu = zeros(n)
    z = zeros(n)

    l[1] = 1
    mu[1] = 0
    z[1] = 0

    for i in 2:(n-1)
        l[i] = 2 * (x[i+1] - x[i-1]) - h[i-1] * mu[i-1]
        mu[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i-1] * z[i-1]) / l[i]
    end

    l[n] = 1
    z[n] = 0
    c[n] = 0

    for j in (n-1):-1:1
        c[j] = z[j] - mu[j] * c[j+1]
        b[j] = (a[j+1] - a[j]) / h[j] - h[j] * (c[j+1] + 2 * c[j]) / 3
        d[j] = (c[j+1] - c[j]) / (3 * h[j])
    end

    return a, b, c, d
end

function createCubicSpline(x::Vector{Float64}, y::Vector{Float64})
    a, b, c, d = calculateCubicSplineCoefficients(x, y)
    return CubicSpline(x, y, a, b, c, d)
end

function evaluate(spline::CubicSpline, x_val::Float64)
    # Find the right place in the array
    i = findlast(spline.x .<= x_val)
    if i == nothing || i == length(spline.x)
        throw(DomainError(x_val, "Value outside of spline range."))
    end

    dx = x_val - spline.x[i]
    return spline.a[i] + spline.b[i] * dx + spline.c[i] * dx^2 + spline.d[i] * dx^3
end



In [ ]:
# Example usage
x = [1.0, 2.0, 3.0, 4.0]
y = [1.0, 4.0, 9.0, 16.0]

spline = createCubicSpline(x, y)
println(evaluate(spline, 2.5))  # Evaluate the spline at x = 2.5


In [ ]:
# Now, the interpolateDataForPoints function
function interpolateDataForPoints5(wanted_rows, wanted_columns, rows, columns, lats, lons, incidenceangle, height, azimuth_time, slant_range_time_seconds, elevation_angle)
    # Create cubic splines for each parameter
    lat_spline = createCubicSpline(rows, lats)
    lon_spline = createCubicSpline(rows, lons)
    incidence_spline = createCubicSpline(rows, incidenceangle)
    height_spline = createCubicSpline(rows, height)
    azimuth_spline = createCubicSpline(rows, azimuth_time)
    slant_range_spline = createCubicSpline(rows, slant_range_time_seconds)
    elevation_spline = createCubicSpline(rows, elevation_angle)

    # Initialize vectors for the interpolated values
    interpolated_lats = Float64[]
    interpolated_lons = Float64[]
    interpolated_incidence = Float64[]
    interpolated_height = Float64[]
    interpolated_azimuth = Float64[]
    interpolated_slant_range = Float64[]
    interpolated_elevation = Float64[]

    # Interpolate for each pair of wanted rows and columns
    for (row, col) in zip(wanted_rows, wanted_columns)
        push!(interpolated_lats, evaluate(lat_spline, row))
        push!(interpolated_lons, evaluate(lon_spline, row))
        push!(interpolated_incidence, evaluate(incidence_spline, row))
        push!(interpolated_height, evaluate(height_spline, row))
        push!(interpolated_azimuth, evaluate(azimuth_spline, row))
        push!(interpolated_slant_range, evaluate(slant_range_spline, row))
        push!(interpolated_elevation, evaluate(elevation_spline, row))
    end

    # Return the results in a dictionary
    return Dict("rows" => wanted_rows, "columns" => wanted_columns, "latitudes" => interpolated_lats, "longitudes" => interpolated_lons, "incidence_angles" => interpolated_incidence, "heights" => interpolated_height, "azimuth_times" => interpolated_azimuth, "slant_range_times" => interpolated_slant_range, "elevation_angles" => interpolated_elevation)
end


In [ ]:
x_values = [1, 2, 3, 4, 5]
y_values = [1, 2, 3, 4, 5]
x, y = 3.1, 3.5
findNearestPoints(x, y, x_values, y_values)

In [ ]:
sentinel1_metadata = test_img.metadata

In [ ]:
rows = sentinel1_metadata.geolocation.lines 
columns = sentinel1_metadata.geolocation.samples

lats = sentinel1_metadata.geolocation.latitude
lons = sentinel1_metadata.geolocation.longitude

incidenceangle = sentinel1_metadata.geolocation.incidence_angle
height = sentinel1_metadata.geolocation.height
azimuth_time = sentinel1_metadata.geolocation.azimuth_time
slant_range_time_seconds = sentinel1_metadata.geolocation.slant_range_time_seconds
elevation_angle = sentinel1_metadata.geolocation.elevation_angle;

In [ ]:
row, colm = 9999, 4010  # 9213, 10529, x y 

findNearestPoints(row, colm,rows, columns)

In [ ]:
"""
    findNearestPoints(x, y, x_values, y_values)

Find the four points forming a bounding box around the point (x, y) in a 2D grid. 
These points are the nearest known points from the provided `x_values` and `y_values` arrays.

# Arguments
- `x`: The x-coordinate of the target point.
- `y`: The y-coordinate of the target point.
- `x_values`: An array of x-coordinates of known points.
- `y_values`: An array of y-coordinates of known points.

# Returns
A tuple of four integers, representing the indices of the points in `x_values` and `y_values` 
that form the bounding box around the point (x, y). The indices are returned in the following order: 
index of left boundary along x-axis, index of right boundary along x-axis, 
index of bottom boundary along y-axis, and index of top boundary along y-axis.

# Example
```julia
x_values = [1, 2, 3, 4, 5]
y_values = [1, 2, 3, 4, 5]
x, y = 2.5, 3.5
findNearestPoints(x, y, x_values, y_values)
"""
function findNearestPoints(column, row, columns, rows)

    x = column 
    y = row 
    x_values = columns
    y_values = rows

    # Find the four points forming a bounding box around (x, y)
    #sorted_x = sortperm(x_values)
    #sorted_y = sortperm(y_values)
    sorted_x = x_values
    sorted_y = y_values

    #x1_idx = findlast(x_values .<= x)
    #x2_idx = findfirst(x_values .>= x)
    #y1_idx = findlast(y_values .<= y)
    #y2_idx = findfirst(y_values .>= y)#

    x1_idx = findlast(x_values .<= x)
    x2_idx = findfirst(x_values.>= x)
    y1_idx = findlast(y_values.<= y)
    y2_idx = findfirst(y_values .>= y)

    # Handle edge cases
    x1_idx = x1_idx === nothing ? 1 : x1_idx
    x2_idx = x2_idx === nothing ? length(x_values) : x2_idx
    y1_idx = y1_idx === nothing ? 1 : y1_idx
    y2_idx = y2_idx === nothing ? length(y_values) : y2_idx

    return sorted_x[x1_idx], sorted_x[x2_idx], sorted_y[y1_idx], sorted_y[y2_idx]
end

In [ ]:
columns[5]

In [ ]:
rows[105]

In [ ]:
rows[106]

In [ ]:
column, row, columns, rows

In [ ]:
columns[108]

In [ ]:
columns

In [ ]:
"""
    findNearestPoints(x, y, x_values, y_values)

Find the four points forming a bounding box around the point (x, y) in a 2D grid. 
These points are the nearest known points from the provided `x_values` and `y_values` arrays.

# Arguments
- `x`: The x-coordinate of the target point.
- `y`: The y-coordinate of the target point.
- `x_values`: An array of x-coordinates of known points.
- `y_values`: An array of y-coordinates of known points.

# Returns
A tuple of four integers, representing the indices of the points in `x_values` and `y_values` 
that form the bounding box around the point (x, y). The indices are returned in the following order: 
index of left boundary along x-axis, index of right boundary along x-axis, 
index of bottom boundary along y-axis, and index of top boundary along y-axis.

# Example
```julia
x_values = [1, 2, 3, 4, 5]
y_values = [1, 2, 3, 4, 5]
x, y = 2.5, 3.5
findNearestPoints(x, y, x_values, y_values)
"""
function findNearestPoints(column, row, columns, rows)

    x = column 
    y = row 
    x_values = columns
    y_values = rows

    # Find the four points forming a bounding box around (x, y)
    #sorted_x = sortperm(x_values)
    #sorted_y = sortperm(y_values)
    sorted_x = x_values
    sorted_y = y_values

    #x1_idx = findlast(x_values .<= x)
    #x2_idx = findfirst(x_values .>= x)
    #y1_idx = findlast(y_values .<= y)
    #y2_idx = findfirst(y_values .>= y)#

    x1_idx = findlast(x_values[sorted_x] .<= x)
    x2_idx = findfirst(x_values[sorted_x] .>= x)
    y1_idx = findlast(y_values[sorted_y] .<= y)
    y2_idx = findfirst(y_values[sorted_y] .>= y)

    # Handle edge cases
    x1_idx = x1_idx === nothing ? 1 : x1_idx
    x2_idx = x2_idx === nothing ? length(x_values) : x2_idx
    y1_idx = y1_idx === nothing ? 1 : y1_idx
    y2_idx = y2_idx === nothing ? length(y_values) : y2_idx

    return sorted_x[x1_idx], sorted_x[x2_idx], sorted_y[y1_idx], sorted_y[y2_idx]
end

function bilinearInterpolation(x, y, x_values, y_values, z_values)
    x1_idx, x2_idx, y1_idx, y2_idx = findNearestPoints(x, y, x_values, y_values)

    # Extract the coordinates of the four points
    x1, x2 = x_values[x1_idx], x_values[x2_idx]
    y1, y2 = y_values[y1_idx], y_values[y2_idx]

    # Values at the corners of the rectangle
    Q11 = z_values[x1_idx]
    Q12 = z_values[y1_idx]
    Q21 = z_values[x2_idx]
    Q22 = z_values[y2_idx]

    # Bilinear interpolation formula
    fxy1 = ((x2 - x) / (x2 - x1)) * Q11 + ((x - x1) / (x2 - x1)) * Q21
    fxy2 = ((x2 - x) / (x2 - x1)) * Q12 + ((x - x1) / (x2 - x1)) * Q22
    fxy = ((y2 - y) / (y2 - y1)) * fxy1 + ((y - y1) / (y2 - y1)) * fxy2

    return fxy
end

function interpolateDataForPoints9(wanted_rows, wanted_columns, rows, columns, lats, lons, incidenceangle, height, azimuth_time, slant_range_time_seconds, elevation_angle)
    interpolated_lats = Float64[]
    interpolated_lons = Float64[]
    interpolated_incidence = Float64[]
    interpolated_height = Float64[]
    interpolated_azimuth = Float64[]
    interpolated_slant_range = Float64[]
    interpolated_elevation = Float64[]

    for (row, col) in zip(wanted_rows, wanted_columns)
        push!(interpolated_lats, bilinearInterpolation(row, col, rows, columns, lats))
        push!(interpolated_lons, bilinearInterpolation(row, col, rows, columns, lons))
        push!(interpolated_incidence, bilinearInterpolation(row, col, rows, columns, incidenceangle))
        push!(interpolated_height, bilinearInterpolation(row, col, rows, columns, height))
        push!(interpolated_azimuth, bilinearInterpolation(row, col, rows, columns, azimuth_time))
        push!(interpolated_slant_range, bilinearInterpolation(row, col, rows, columns, slant_range_time_seconds))
        push!(interpolated_elevation, bilinearInterpolation(row, col, rows, columns, elevation_angle))
    end

    return Dict("rows" => wanted_rows, "columns" => wanted_columns, "latitudes" => interpolated_lats, "longitudes" => interpolated_lons, "incidence_angles" => interpolated_incidence, "heights" => interpolated_height, "azimuth_times" => interpolated_azimuth, "slant_range_times" => interpolated_slant_range, "elevation_angles" => interpolated_elevation)
end


In [ ]:
function interpolateLatLonForPointsSentinel1(wanted_rows, wanted_columns, sentinel1_metadata)
    rows = sentinel1_metadata.geolocation.lines 
    columns = sentinel1_metadata.geolocation.samples

    lats = sentinel1_metadata.geolocation.latitude
    lons = sentinel1_metadata.geolocation.longitude

    incidenceangle = sentinel1_metadata.geolocation.incidence_angle
    height = sentinel1_metadata.geolocation.height
    azimuth_time = sentinel1_metadata.geolocation.azimuth_time
    slant_range_time_seconds = sentinel1_metadata.geolocation.slant_range_time_seconds
    elevation_angle = sentinel1_metadata.geolocation.elevation_angle


    return interpolateDataForPoints9(wanted_rows, wanted_columns, rows, columns, lats, lons, incidenceangle, height, azimuth_time, slant_range_time_seconds, elevation_angle)
end



In [ ]:
#geoarray = GeoArrays.read("/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE/measurement/s1a-iw-grd-vv-20220612t173329-20220612t173354-043633-05359a-001.tiff",
#                        , masked=false, band=1)


In [ ]:
#                        , masked=false, band=1) # remove this

In [ ]:
Test_name.Visualize.sar2gray(test_img.data[1][13000:13500,19300:20100],p_quantile=0.99)

In [ ]:
coords_and_stuff = interpolateLatLonForPointsSentinel1(columns,rows,test_img.metadata)

In [ ]:
coords_and_stuff = interpolateLatLonForPointsSentinel1(rows,columns,test_img.metadata)

In [ ]:
#coords_and_stuff = Test_name.Sensors.Sentinel1.interpolateLatLonForPointsSentinel1(rows,columns,test_img.metadata)

In [ ]:
fieldnames(Test_name.Sensors.Sentinel1.Sentinel1GeolocationGrid)

In [ ]:
import ONNXRunTime as OX
#using FLoops

In [ ]:
model = OX.load_inference("/Users/kaaso/Documents/coding/JuliaEO2024/src/ship_detector/model/yolov8_ship_k5_more_parms_nms.onnx")

In [ ]:
test_img

In [ ]:
th = test_img.data[2][13000:13000+256*2-1,19300:19300+256*2-1];
th = round.(Int64,(th./maximum(th)).*255);
s = cat(th, th, th, dims=3)
s = Float32.(permutedims(s, [3, 1, 2]));
s = reshape(s, 1, 3, size(th)[1], size(th)[2])

inpu = Dict("images" => s)

In [ ]:
using Images
using Images, FileIO


In [ ]:
res = model(inpu)

In [ ]:
res["output0"]

In [ ]:
a = []
ix = []
prob = []
x = []
y = []
width = []
height = []


for i in 1:size(res["output0"][1,1,:])[1]
    if res["output0"][1,5,i] > 0.01
        #push!(a, (i,res["output0"][1,:,i]))
        push!(a, (i,round.(Int64,res["output0"][1,1:4,i]),Float64(res["output0"][1,5,i])))
        push!(ix,i)
        push!(prob, Float64(res["output0"][1,5,i]))
        push!(x, round(Int64,res["output0"][1,1,i]))   
        push!(y, round(Int64,res["output0"][1,2,i]))   
        push!(width, round(Int64,res["output0"][1,3,i]))   
        push!(height, round(Int64,res["output0"][1,4,i]))   
        
    end
end

resutls = Dict(
    "index" => ix,
    "probability" => prob,
    "x" => x,
    "y" => y,
    "width" => width,
    "height" => height
)

In [ ]:
resutls = Dict(
    "index" => ix,
    "probability" => prob,
    "x" => x,
    "y" => y,
    "width" => width,
    "height" => height
)

println(size(y))

In [ ]:

# Example usage
# image_matrix = <your 1536x1536 matrix>
# bbox_dict = <your dictionary with bounding box details>
plot_image_with_boxes2(th, resutls)

In [ ]:
using Plots, Statistics

function plot_image_with_boxes2(image_matrix, bbox_dict,p_quantile=0.99)

    min_value = minimum(reshape(image_matrix,:))
    factor = quantile(reshape(image_matrix,:),p_quantile) - min_value

    clamp_value = quantile(vec(image_matrix), p_quantile)
    clamped_image_matrix = clamp.(image_matrix, min(image_matrix...), clamp_value)

    image_matrix = (image_matrix.-min_value)./factor


    # Create a plot of the image
    img_plot = heatmap(clamped_image_matrix, c=:grays, axis=false)

    # Extract bounding box details
    xs = bbox_dict["x"]
    ys = bbox_dict["y"]
    widths = bbox_dict["width"]
    heights = bbox_dict["height"]

    # Plot each bounding box
    for i in 1:length(xs)
        x_center = xs[i]
        y_center = ys[i]
        width = widths[i]
        height = heights[i]

        # Calculate the corners of the rectangle
        x_left = x_center - width / 2
        y_bottom = y_center - height / 2

        # Draw the rectangle
        plot!(img_plot, [x_left, x_left + width, x_left + width, x_left, x_left], 
                      [y_bottom, y_bottom, y_bottom + height, y_bottom + height, y_bottom], 
                      seriestype=:shape, linecolor=:red, fillalpha=0)
    end

    # Display the plot
    display(img_plot)
end



In [ ]:
# specify the path to your local image file
img_path = "/Users/kaaso/Documents/coding/JuliaEO2024/0a0824ed-e2f9-47f4-bd59-ee53f9c399eb.png"
img = load(img_path)

In [ ]:
size(in)

In [ ]:
function get_input_array(img)

    resized = imresize(img, (256, 256))
    
    mat = channelview(resized)
    
    original_array = Float32.(mat)
    original_array = cat(original_array, original_array, original_array, dims=3)
    return reshape(original_array, (1,size(original_array)...))
    
 end

In [ ]:
res = model(in)

In [ ]:
#x,y,width,height,

In [ ]:
a

In [ ]:
Test_name.Visualize.sar2gray(th[345:360,70:90],p_quantile=0.99)

In [ ]:
size(res["output0"])

In [ ]:
typeof(res["output0"])

In [ ]:
function filter_array(arr::Array{Float64, 3})
    # Check if the size of the second dimension is at least 1
    if size(arr, 2) < 1
        error("The array does not have the required dimensions.")
    end

    # Filter the array based on the condition
    filtered = [arr[:, i, :] for i in 1:size(arr, 2) if arr[:, i, end] > 0.6]

    # Concatenate the filtered elements back into an array
    return cat(filtered..., dims=2)
end

In [ ]:
a

In [ ]:
(1,res["output0"][1,:,2])

In [ ]:
size(res["output0"][1,1,:])

In [ ]:
push!(a,(1,3,2))
push!(a,(1,3,2))

In [ ]:
size(res["output0"])

In [ ]:
res["output0"][1,:,1]

In [ ]:
filter_array(res)

In [ ]:
# Create a boolean mask for the condition
mask = res[:, 5, :] .> 0.6


In [ ]:

# Use broadcasting to apply the mask to the entire array
filtered_array = res[:, 5, :] .* ma